In [1]:
import numpy as np
import time
import pandas as pd
from pathlib import Path
import sys
import json
import datetime
from pre_process_tetrode import get_process_save_tetrode


/Users/alexgonzalez/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# make sure you are processing the right tasks by looking at the file.
AnimalID = 'Li'
date_obj = datetime.date.today()
date_str= "%s_%s_%s" % (date_obj.month,date_obj.day,date_obj.year)

TaskDir = Path('../PreProcessing/TasksDir/')

with open(str(TaskDir) + '/PreProcessingTable_{}_{}.json'.format(AnimalID,date_str), 'r') as f:
        task_table = json.load(f)

In [3]:
task_table[str(3)]

{'filename': ['CSC1a.ncs', 'CSC1b.ncs', 'CSC1c.ncs', 'CSC1d.ncs'],
 'filepath': '/Users/alexgonzalez/Google Drive/PostDoc/Data/T3g/Li/Li_T3g_060418',
 'savepath': '/Users/alexgonzalez/Google Drive/PostDoc/Data/T3g/Li/Li_T3g_060418_Results',
 'tt_id': '1',
 'type': 'tt'}

In [13]:
fp = Path(task['filepath'])
csc = task['filename']

NameError: name 'Path' is not defined

In [5]:
(fp/csc[0]).exists()

True

In [10]:
fp

NameError: name 'fp' is not defined

0.41354166666666664